###Demonstration of lanchain chain components before and after implementing runnnables

In [1]:
import random

In [5]:
class FakeLLM:
  def __init__(self):
    print('LLM created')

  def predict(self, prompt):
    response_list = [
        'Delhi is the capital of India',
        'AI stands for Artificial Intelligence',
        'IPL stands for Indian Premier League'
    ]

    return {'response': random.choice(response_list)}

In [6]:
llm = FakeLLM()

LLM created


In [10]:
class FakePromptTemplate:
  def __init__(self, template, input_variables):
    self.template = template
    self.input_variables = input_variables
    print('PromptTemplate created')

  def format(self, input_dict):
    return self.template.format(**input_dict)
    print('PromptTemplate created')

In [15]:
template = FakePromptTemplate(template="Write a {length} poem about {topic}", input_variables=['topic'])

PromptTemplate created


In [17]:
prompt = template.format({'length':'short', 'topic': 'AI'})

In [18]:
llm.predict(prompt)

{'response': 'IPL stands for Indian Premier League'}

In [20]:
class FakeLLMChain:
  def __init__(self, llm, prompt):
    self.llm = llm
    self.prompt = prompt

  def run(self, input_dict):
    prompt = self.prompt.format(input_dict)
    result = self.llm.predict(prompt)
    return result['response']

In [21]:
template = FakePromptTemplate(template="Write a {length} poem about {topic}", input_variables=['topic'])
llm = FakeLLM()

PromptTemplate created
LLM created


In [22]:
chain = FakeLLMChain(llm, template)

In [23]:
chain.run({'length': 'short', 'topic': 'india'})

'Delhi is the capital of India'

### understanding runnable by creating our own sample


In [24]:
from abc import ABC, abstractmethod

In [25]:
class Runnable(ABC):
  @abstractmethod
  def invoke(input_data):
    pass

In [26]:
class FakeLLM(Runnable):
  def __init__(self):
    print('LLM created')

  def invoke(self, prompt):
    response_list = [
        'Delhi is the capital of India',
        'AI stands for Artificial Intelligence',
        'IPL stands for Indian Premier League'
    ]

    return {'response': random.choice(response_list)}

  def predict(self, prompt):
    response_list = [
        'Delhi is the capital of India',
        'AI stands for Artificial Intelligence',
        'IPL stands for Indian Premier League'
    ]

    return {'response': random.choice(response_list)}

In [27]:
class FakePromptTemplate(Runnable):
  def __init__(self, template, input_variables):
    self.template = template
    self.input_variables = input_variables
    print('PromptTemplate created')

  def invoke(self, input_dict):
    return self.template.format(**input_dict)

  def format(self, input_dict):
    return self.template.format(**input_dict)
    print('PromptTemplate created')

In [28]:
template = FakePromptTemplate(template="Write a {length} poem about {topic}", input_variables=['topic'])
llm = FakeLLM()

PromptTemplate created
LLM created


In [33]:
class FakeStrOutputParser(Runnable):
  def __init__(self):
    print('OutputParser created')

  def invoke(self, input_data):
    return input_data['response']

In [34]:
class RunnableConnector(Runnable):
  def __init__(self, runnable_list):
    self.runnable_list = runnable_list

  def invoke(self, input_data):
    output = input_data
    for runnable in self.runnable_list:
      runnables_output = runnable.invoke(output)
      output = runnables_output
    return output

In [35]:
parser = FakeStrOutputParser()

OutputParser created


In [36]:
chain = RunnableConnector([template, llm, parser])

In [37]:
chain.invoke({'length': 'long', 'topic': 'India'})

'Delhi is the capital of India'